# Result-Dependent if/else

task result dependent if/else statements inside lattices (B)

In case you'd like to use some conditional logic in your workflow, say, something like method (1), it'll be better if you introduced it in the resulting electron directly instead of inside the lattice, something like method (2). The reason for that is that during the workflow dependency graph creation, the return values of the `task` functions is unknown, thus these results are of the type `Electron`. This is because the execution doesn't happen, and thus the dependency of logic on the result of the task cannot be relied upon.


In [ ]:
import covalent as ct

# Method (1):
@ct.electron
def task_1(x):
    return x * 2

@ct.electron
def task_2(x):
    return x ** 3

@ct.lattice
def workflow(a):
    
    res_1 = task_1(a)

    if res_1 == 10:
        final_res = task_2(res_1)
    else:
        final_res = res_1
    
    return final_res


# Method (2):
@ct.electron
def task_2_new(x):
    if x == 10:
        return x ** 3
    else:
        return x

@ct.lattice
def workflow_2(a):
    res_1 = task_1(a)
    return task_2_new(res_1)